# Extracting all transactions data from all the credit card statements till date.

## Imports

In [1]:
import PyPDF2
import re
import pandas as pd
from pathlib import Path
from pandas import ExcelWriter


In [2]:
# PDF password
password = "XXXXXXXX"

# regex used for Axis Bank Flipkart Credit Card. Might need little tweak for other cards
regex_dict = {
                "Total Payment Due":          (r"\d{1,2}\,\d{3}.\d{2}\s*[DC]r|\d+\.\d{2}\s*[DC]r?", 0),
                "Minimum Payment Due":        (r"\d{1,2}\,\d{3}.\d{2}\s*[DC]r|\d+\.\d{2}\s*[DC]r?", 1),
                "Statement Period":           (r"\d{2}\/\d{2}\/\d{4} - \d{2}\/\d{2}\/\d{4}", 0),
                "Payment Due Date":           (r"\d{2}\/\d{2}\/\d{4}", 2),
                "Statement Generation Date":  (r"\d{2}\/\d{2}\/\d{4}", 3),
                "Credit Card Number":         (r"\d{6}\*{6}\d{4}", 0),
                "Credit Limit":               (r"\d{1,2}\,\d{3}.\d{2}|\d+\.\d{2}?", 2),
                "Available Credit Limit":     (r"\d{1,2}\,\d{3}.\d{2}|\d+\.\d{2}?", 3),
                "Available Cash Limit":       (r"\d{1,2}\,\d{3}.\d{2}|\d+\.\d{2}?", 4),
                "Previous Balance":           (r"\d{1,2}\,\d{3}.\d{2}|\d+\.\d{2}?", 5),
                "Payments":                   (r"\d{1,2}\,\d{3}.\d{2}|\d+\.\d{2}?", 6),
                "Credits":                    (r"\d{1,2}\,\d{3}.\d{2}|\d+\.\d{2}?", 7),
                "Purchase":                   (r"\d{1,2}\,\d{3}.\d{2}|\d+\.\d{2}?", 8),
                "Cash Advance":               (r"\d{1,2}\,\d{3}.\d{2}|\d+\.\d{2}?", 9),
                "Other Debit&Charges":        (r"\d{1,2}\,\d{3}.\d{2}|\d+\.\d{2}?", 10),
                "=Total Payment Due":         (r"\d{1,2}\,\d{3}.\d{2}\s*[DC]r|\d+\.\d{2}\s*[DC]r?", 0),
            }

In [3]:
def extract_table(content):
    
    # New Data Frame
    df_tab = pd.DataFrame()

    # Dates for the transactions
    regex_stat_per = regex_dict['Payment Due Date'][0]
    matches = re.findall(regex_stat_per, content, re.MULTILINE)
    df_tab['DATE'] = matches[4:]
    df_tab.head()

    # Amount and Cashback 
    regex_stat_per = regex_dict['Total Payment Due'][0]
    matches = re.findall(regex_stat_per, content, re.MULTILINE)
    matches = matches[-(df_tab.shape[0]*2):]

    AMOUNT_list = []
    CASHBACK_list = []

    for i in range(0, (df_tab.shape[0]*2)):
        if i%2 == 0:
            AMOUNT_list.append(matches[i])
        else:
            CASHBACK_list.append(matches[i])

    df_tab['AMOUNT'] = AMOUNT_list
    df_tab['CASHBACK EARNED'] = CASHBACK_list
    df_tab.head()

    # Text Part in Table
    regex_stat_per = r'\n\d{2}\/\d{2}\/\d{4}\s(\w.*?\s\s*)?[\d]+'
    matches = re.findall(regex_stat_per, content, re.MULTILINE)

    df_tab['TRANSACTION DETAILS'] = matches
    
    return df_tab

## Header Info

In [2]:

all_credit_header = []

# iterating all the files with pdf extension in current folder(".")
for path in Path(".").glob("*.pdf"):
    
    pdf = open(path, 'rb')
    pdf = PyPDF2.PdfFileReader(pdf)
    pdf.decrypt(password=password)

    # Adding all pages content
    content = ''
    for i in range(0, pdf.getNumPages()): 
        content += pdf.getPage(i).extractText()
        content += '\n'

    content = content[:-1]
    
    # getting value for each field from content using regex
    header_list = []
    for key, val in regex_dict.items():
        matches = re.findall(val[0], content, re.MULTILINE)
        header_list.append(matches[val[1]])
        # print(key, "-------------------------->", matches[val[1]])

    all_credit_header.append(header_list)
    

df = pd.DataFrame(data = all_credit_header, columns = regex_dict.keys())
df.loc[:, 'Statement Generation Date'] = pd.to_datetime(df['Statement Generation Date'], format='%d/%m/%Y')
df.sort_values('Statement Generation Date')
df.head()

## Table Info

In [3]:
df_list = []

# iterating all the files with pdf extension in current folder(".")
for path in Path(".").glob("*.pdf"):
    
    pdf = open(path, 'rb')
    pdf = PyPDF2.PdfFileReader(pdf)
    pdf.decrypt(password=password)

    # Adding all pages content
    content = ''
    for i in range(0, pdf.getNumPages()): 
        content += pdf.getPage(i).extractText()
        content += '\n'

    content = content[:-1]
    
    df_one = extract_table(content)
    df_one['File'] = path
    df_list.append(df_one)
    
df_final = pd.concat(df_list)
df_final.head()
    

In [7]:
writer = ExcelWriter('All Transactions.xlsx')

df_final.to_excel(writer, index=False, sheet_name='Detailed')
df.to_excel(writer, index=False, sheet_name='Header')

writer.save()

# Note: 
-     This code is very specific to Axis FLipkart Credit Card.